# 🚀 Arabic Documents Summarization, NER & Topic Modeling

## Overview
This comprehensive Arabic NLP pipeline combines multiple state-of-the-art models for:
- **Text Summarization** (Extractive & Abstractive)
- **Named Entity Recognition (NER)** with multiple model comparison
- **Sentiment Analysis**
- **Topic Modeling** with coherence metrics

### Key Features
✅ Multi-model comparison for fair benchmarking  
✅ Automatic evaluation metrics (ROUGE, F1, Coherence)  
✅ Support for both extractive and abstractive summarization  
✅ Arabic text preprocessing and morphological analysis  
✅ Real-world dataset with annotated entities  

## Installation & Setup
```bash
# Install core dependencies
pip install camel-tools scikit-learn networkx numpy transformers torch nltk

# Download CAMeL Tools data
camel_data -i ner-arabert                    # NER (541 MB)
camel_data -i sentiment-analysis-arabert     # Sentiment (541 MB)
camel_data -i morphology-db-msa-r13          # Morphology (40 MB)
camel_data -i disambig-mle-calima-msa-r13    # Disambiguation (88 MB)
```

In [ ]:
"""
🔧 SETUP: Install dependencies and initialize NLTK data
This cell downloads all required models and initializes the environment.
"""

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import re
import warnings
import numpy as np
import torch
import nltk

# =============================================
# 🛠️ SETUP & IMPORTS
# =============================================
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    print("⬇️ Downloading NLTK data...")
    nltk.download('punkt', quiet=True)
    nltk.download('punkt_tab', quiet=True)

from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM, 
    AutoModelForTokenClassification,
    pipeline as hf_pipeline
)

# CAMeL Tools
from camel_tools.utils.normalize import normalize_unicode, normalize_alef_ar, normalize_alef_maksura_ar, normalize_teh_marbuta_ar
from camel_tools.utils.dediac import dediac_ar
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.analyzer import Analyzer
from camel_tools.sentiment import SentimentAnalyzer

# Optional Libraries
try:
    import sumy
    from sumy.parsers.plaintext import PlaintextParser
    from sumy.nlp.tokenizers import Tokenizer as SumyTokenizer
    from sumy.summarizers.lex_rank import LexRankSummarizer
    from sumy.summarizers.text_rank import TextRankSummarizer
    from sumy.summarizers.lsa import LsaSummarizer
    SUMY_AVAILABLE = True
except ImportError:
    SUMY_AVAILABLE = False

try:
    import gensim
    from gensim import corpora
    from gensim.models import LdaModel
    from gensim.models.coherencemodel import CoherenceModel
    GENSIM_AVAILABLE = True
except ImportError:
    GENSIM_AVAILABLE = False

try:
    import stanza
    STANZA_AVAILABLE = True
except ImportError:
    STANZA_AVAILABLE = False

warnings.filterwarnings('ignore')
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE_ID = 0 if torch.cuda.is_available() else -1

print(f"✅ Device: {DEVICE}")
print(f"✅ Models: Sumy={SUMY_AVAILABLE}, Gensim={GENSIM_AVAILABLE}, Stanza={STANZA_AVAILABLE}")

🚀 ARABIC NLP PIPELINE: BENCHMARK EDITION
  📚 Loading CAMeL Morphology...
  🏷️ Loading NER Models...


Some weights of the model checkpoint at /Users/user/.camel_tools/data/ner/arabert were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  📝 Loading Summarization Models...
  📊 Topic Modeling: Gensim

📄 DETAILED ANALYSIS (LARGE DOCS)

📂 Document 1 (148 words)
📝 Summarization:


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


   [AraBART]: اعلنت شركه ارامكو السعوديه، عملاق النفط العالمي واكبر شركه طاقه في العالم من حيث القيمه السوقيه، الي...
🏷️ NER (CAMeL):


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


   Entities found: ارامكو السعودية, ريال, امين حسن الناصر, الظهران, الناصر, توتال انرجيز...
😊 Sentiment:
   True: mixed | Pred: positive

📂 Document 2 (88 words)
📝 Summarization:
   [AraBART]: اختتمت القمه العربيه الطارئه اعمالها في العاصمه الاردنيه عمان، وسط حضور رفيع المستوي من قاده الدول ا...
🏷️ NER (CAMeL):
   Entities found: عمان, عبدالله الثاني, عبدالله, محمد بن سلمان, عبدالفتاح السيسي, سوريا...
😊 Sentiment:
   True: neutral | Pred: neutral

📂 Document 3 (87 words)
📝 Summarization:
   [AraBART]: تشهد المملكه العربيه السعوديه طفره تقنيه هائله، حيث اعلنت جامعه الملك عبدالله للعلوم والتقنيه (كاوست...
🏷️ NER (CAMeL):
   Entities found: المملكة العربية السعودية, جامعة الملك عبدالله للعلوم والتقنية, كاوست, جوجل, ومايكروسوفت, الملك فيصل التخصصي...
😊 Sentiment:
   True: positive | Pred: positive

🏆 FINAL BENCHMARK SCORES

📝 SUMMARIZATION (ROUGE-1)
  mT5-XLSum       : 0.2186
  Sumy-LexRank    : 0.1873
  Sumy-LSA        : 0.1873
  AraBART         : 0.1844

🏷️ NER (F1 Score)
  CAMeL        

## Components & Models

### 1. **Text Preprocessing & Normalization**
- Unicode normalization
- Diacritical mark removal (Tasdeed)
- Alef/Maksura/Teh Marbuta standardization
- Stopword removal
- Lemmatization using CAMeL morphological database

### 2. **Named Entity Recognition (NER)**
Comparison of 3 models:
- **CAMeL Tools (AraBERT)** - Pre-trained on Arabic
- **Hatmimoha (BERT)** - BERT-based Arabic NER
- **Stanford Stanza** - Multilingual NLP pipeline

Entity Types: `PERSON` (PERS), `LOCATION` (LOC), `ORGANIZATION` (ORG), `MISCELLANEOUS` (MISC)

### 3. **Text Summarization**
**Extractive Methods:**
- Sumy-LexRank
- Sumy-LSA
- Sumy-TextRank

**Abstractive Methods:**
- mT5-XLSum (multilingual)
- AraBART (Arabic-specific)

### 4. **Sentiment Analysis**
- CAMeL Tools Sentiment Analyzer
- Labels: Positive, Negative, Neutral

### 5. **Topic Modeling**
- Latent Dirichlet Allocation (LDA)
- Coherence evaluation (C_V score)
- Automatic topic extraction

---

## Metrics & Evaluation

### ROUGE Score (Summarization)
- **ROUGE-1**: Unigram overlap between reference and hypothesis
- **Precision**: Overlap / Hypothesis length
- **Recall**: Overlap / Reference length
- **F1**: Harmonic mean of precision and recall

### NER Metrics
- **Precision**: Correct entities / Total predicted entities
- **Recall**: Correct entities / Total reference entities
- **F1 Score**: Harmonic mean (with partial matching support)

### Topic Modeling Metrics
- **Coherence (C_V)**: Measures semantic coherence of topics (0-1, higher is better)

### Sentiment Accuracy
- **Accuracy**: Percentage of correctly predicted sentiment labels

---

## Class Architecture

### 1. `EvaluationMetrics`
Provides all evaluation functions:
- `normalize_arabic()`: Normalize Arabic text for fair comparison
- `rouge_scores()`: Calculate ROUGE-1 F1 score
- `ner_metrics()`: Calculate NER precision, recall, F1

### 2. `ArabicPreprocessor`
Handles all text preprocessing:
- Loads CAMeL morphological database
- Normalizes Unicode and diacriticals
- Tokenizes and lemmatizes text
- Removes stopwords

### 3. `ArabicNER`
Multi-model NER comparison:
- Loads all 3 models (CAMeL, Hatmimoha, Stanza)
- Provides `extract_all()` for parallel extraction
- Standardizes output format

### 4. `ArabicSummarizer`
Combines extractive and abstractive methods:
- Loads Sumy models (LexRank, LSA)
- Loads neural models (AraBART, mT5-XLSum)
- Gracefully handles missing models

### 5. `TopicModeler`
LDA-based topic extraction:
- Preprocesses documents
- Builds vocabulary and corpus
- Trains LDA model with 3 topics
- Calculates coherence score

### 6. `UltimatePipeline`
Orchestrates the entire workflow:
- Initializes all components
- Runs analysis on datasets
- Aggregates and reports results

---

## Running the Pipeline

Execute the following cell to run the complete benchmark:

```python
pipeline = UltimatePipeline()
pipeline.run(get_large_data())
```

This will:
1. Initialize all NLP models (may take 1-2 minutes for first run)
2. Preprocess 3 sample Arabic documents
3. Run NER, summarization, sentiment, and topic modeling
4. Display detailed results for each document
5. Print aggregated benchmark scores

---

In [ ]:
"""
Execute the complete Arabic NLP pipeline with benchmark evaluation
"""

if __name__ == "__main__":
    pipeline = UltimatePipeline()
    pipeline.run(get_large_data())